# SLF

### Acknowledgment
This project includes code adapted from [Surface-Laplacian](https://github.com/alberto-ara/Surface-Laplacian/tree/master),
created by Alberto Ara and licensed under the MIT License.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import codecs, json
from tqdm import tqdm
import numpy as np
from datetime import datetime
import time
from scipy import special
import math

import sys
sys.path.append('drive/MyDrive/BmiResearch')

from constants import constants
from utils.debugger import logger

In [ ]:
output_path = f'{constants.PREPROCESSED_DATASET_PATH}/SLF'

In [ ]:
def surface_laplacian(epochs, m=4, leg_order=50, smoothing=1e-5,):
    # Part of this code is adapted from [alberto-ara/Surface-Laplacian] (https://github.com/alberto-ara/Surface-Laplacian/tree/master)
    # Copyright (c) 2017 Alberto Ara
    # Licensed under the MIT License (https://github.com/alberto-ara/Surface-Laplacian/blob/master/LICENSE)

    montage = {'F5': np.array([-63.058,  54.038,  18.126]), 'F3': np.array([-48.2  ,  57.551,  39.87 ]),
               'Fz': np.array([-0.   , 60.738, 59.463]), 'F4': np.array([48.143, 57.584, 39.892]),
               'F6': np.array([63.045, 54.026, 18.208]), 'FC3': np.array([-59.275,  30.955,  52.471]),
               'FC1': np.array([-32.351,  32.436,  71.598]), 'FC2': np.array([32.351, 32.436, 71.598]),
               'FC4': np.array([59.275, 30.955, 52.471]), 'C5': np.array([-8.0832e+01,  4.9495e-15,  2.6292e+01]),
               'C3': np.array([-6.3171e+01,  3.8681e-15,  5.6872e+01]), 'C1': np.array([-3.4537e+01,  2.1148e-15,  7.7667e+01]),
               'Cz': np.array([-0.0000e+00,  5.2047e-15,  8.5000e+01]), 'C2': np.array([3.4609e+01, 2.1192e-15, 7.7635e+01]),
               'C4': np.array([6.3167e+01, 3.8679e-15, 5.6876e+01]), 'C6': np.array([8.0832e+01, 4.9495e-15, 2.6292e+01]),
               'CP5': np.array([-76.247, -28.763,  24.167]), 'CP3': np.array([-59.275, -30.955,  52.471]),
               'CP1': np.array([-32.351, -32.436,  71.598]), 'CPz': np.array([ 4.0325e-15, -3.2928e+01,  7.8363e+01]),
               'CP2': np.array([ 32.351, -32.436,  71.598]), 'CP4': np.array([ 59.275, -30.955,  52.471]),
               'CP6': np.array([ 76.247, -28.763,  24.167]), 'P5': np.array([-63.058, -54.038,  18.126]),
               'P3': np.array([-48.2  , -57.551,  39.87 ]), 'Pz': np.array([ 7.4383e-15, -6.0738e+01,  5.9463e+01]),
               'P4': np.array([ 48.143, -57.584,  39.892]), 'P6': np.array([ 63.045, -54.026,  18.208]),
               'PO3': np.array([-31.483, -76.153,  20.847]), 'PO4': np.array([ 31.483, -76.153,  20.847]),
               'O1': np.array([-26.133 , -80.784 ,  -4.0011]), 'Oz': np.array([ 1.0407e-14, -8.4981e+01, -1.7860e+00]),
               'O2': np.array([ 26.133 , -80.784 ,  -4.0011])}


    # get electrodes positions
    locs = np.array(list(montage.values()))

    x = locs[:, 0]
    y = locs[:, 1]
    z = locs[:, 2]

    # arrange data
    #     print(epochs.shape)
    data = epochs.T  # np.rollaxis(epochs, 0, 3)
    orig_data_size = np.squeeze(data.shape)

    numelectrodes = len(x)

    # normalize cartesian coordenates to sphere unit
    def cart2sph(x, y, z):
        hxy = np.hypot(x, y)
        r = np.hypot(hxy, z)
        el = np.arctan2(z, hxy)
        az = np.arctan2(y, x)
        return az, el, r

    junk1, junk2, spherical_radii = cart2sph(x, y, z)
    maxrad = np.max(spherical_radii)
    x = x / maxrad
    y = y / maxrad
    z = z / maxrad

    # compute cousine distance between all pairs of electrodes
    cosdist = np.zeros((numelectrodes, numelectrodes))
    for i in range(numelectrodes):
        for j in range(i + 1, numelectrodes):
            cosdist[i, j] = 1 - (((x[i] - x[j]) ** 2 + (y[i] - y[j]) ** 2 + (z[i] - z[j]) ** 2) / 2)

    cosdist = cosdist + cosdist.T + np.identity(numelectrodes)

    # get legendre polynomials
    legpoly = np.zeros((leg_order, numelectrodes, numelectrodes))
    for ni in range(leg_order):
        for i in range(numelectrodes):
            for j in range(i + 1, numelectrodes):
                # temp = special.lpn(8,cosdist[0,1])[0][8]
                legpoly[ni, i, j] = special.lpn(ni + 1, cosdist[i, j])[0][ni + 1]

    legpoly = legpoly + np.transpose(legpoly, (0, 2, 1))

    for i in range(leg_order):
        legpoly[i, :, :] = legpoly[i, :, :] + np.identity(numelectrodes)

    # compute G and H matrixes
    twoN1 = np.multiply(2, range(1, leg_order + 1)) + 1
    gdenom = np.power(np.multiply(range(1, leg_order + 1), range(2, leg_order + 2)), m, dtype=float)
    hdenom = np.power(np.multiply(range(1, leg_order + 1), range(2, leg_order + 2)), m - 1, dtype=float)

    G = np.zeros((numelectrodes, numelectrodes))
    H = np.zeros((numelectrodes, numelectrodes))

    for i in range(numelectrodes):
        for j in range(i, numelectrodes):

            g = 0
            h = 0

            for ni in range(leg_order):
                g = g + (twoN1[ni] * legpoly[ni, i, j]) / gdenom[ni]
                h = h - (twoN1[ni] * legpoly[ni, i, j]) / hdenom[ni]

            G[i, j] = g / (4 * math.pi)
            H[i, j] = -h / (4 * math.pi)

    G = G + G.T
    H = H + H.T

    G = G - np.identity(numelectrodes) * G[1, 1] / 2
    H = H - np.identity(numelectrodes) * H[1, 1] / 2

    if np.any(orig_data_size == 1):
        data = data[:]
    else:
        data = np.reshape(data, (orig_data_size[0], np.prod(orig_data_size[1:3])))

    # compute C matrix
    Gs = G + np.identity(numelectrodes) * smoothing
    GsinvS = np.sum(np.linalg.inv(Gs), 0)
    dataGs = np.dot(data.T, np.linalg.inv(Gs))
    C = dataGs - np.dot(np.atleast_2d(np.sum(dataGs, 1) / np.sum(GsinvS)).T, np.atleast_2d(GsinvS))
    surf_lap = np.reshape(np.transpose(np.dot(C, np.transpose(H))), orig_data_size)
    return surf_lap.T  # np.rollaxis(surf_lap, 2, 0)


def apply_slf(data_chanks_list_train):
    print('[apply_slf]')

    final_train_set = []

    for chank_df in tqdm(data_chanks_list_train):
        sl_data = surface_laplacian(epochs=chank_df.T)
        # scaled_data = standard_scaling(sl_data)
        final_train_set.append(sl_data.T)
    return np.array(final_train_set)



In [ ]:
output_path

'drive/MyDrive/BmiResearch/data/datasets/preprocessed/a_walk_in_the_park/SLF'

In [ ]:
for subject in sorted(os.listdir(constants.BASE_DATASET_PATH)):
  print(subject)

  experiment_settings = dict()

  experiment_settings['general_params'] = {'low_filter':constants.low_filter,
                                          'high_filter':constants.high_filter,
                                          'frequency':constants.freq,
                                          'minutes_for_test':constants.minutes_for_test,
                                          'window_size':constants.window_size,
                                          'overlap':constants.overlap,
                                          'EEG_CHANNELS':constants.EEG_CHANNELS}
  experiment_settings['signal_processing'] = 'SLF'
  experiment_settings['DateTime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
  experiment_settings['BASE_DATASET_PATH'] = constants.BASE_DATASET_PATH
  experiment_settings['OUTPUT_PATH'] = output_path

  output_path_subject = (f'{output_path}/{subject}')
  os.makedirs(output_path_subject)

  experiment_settings[subject] = dict()

  # Fit SLF
  fit_df = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_fit.json', 'r', encoding='utf-8').read()
  fit_df = json.loads(fit_df)
  fit_df = np.array(fit_df)
  fit_df_slf = surface_laplacian(epochs=fit_df, m=4, leg_order=50, smoothing=1e-5)

  fit_df_slf = fit_df_slf.tolist()
  json.dump(fit_df_slf, codecs.open(f'{output_path_subject}/X_fit.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

  # process test
  chanks_test = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_test_chunks.json', 'r', encoding='utf-8').read()
  chanks_test = json.loads(chanks_test)
  chanks_test = np.array(chanks_test)

  starttime = time.perf_counter()
  chanks_test_slf = apply_slf(chanks_test)
  slf_proc_end = time.perf_counter() - starttime
  slf_proc_1ch_s = round(slf_proc_end / chanks_test.shape[0], 5)
  experiment_settings[subject]['proc_1_test_ch_s'] = slf_proc_1ch_s
  experiment_settings[subject]['proc_test_s'] = round(slf_proc_end, 5)
  experiment_settings[subject]['len_test'] = chanks_test.shape[0]
  experiment_settings[subject]['output_path_subject'] = output_path_subject

  chanks_test_slf = chanks_test_slf.tolist()
  json.dump(chanks_test_slf, codecs.open(f'{output_path_subject}/X_test_chunks.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

  # process train
  chanks_train = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_train_chunks.json', 'r', encoding='utf-8').read()
  chanks_train = json.loads(chanks_train)
  chanks_train = np.array(chanks_train)
  chanks_train_slf = apply_slf(chanks_train)

  chanks_train_slf = chanks_train_slf.tolist()
  json.dump(chanks_train_slf, codecs.open(f'{output_path_subject}/X_train_chunks.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)

  json.dump(experiment_settings, codecs.open(f'{output_path_subject}/experiment_settings.json', 'w', encoding='utf-8'),
    separators=(',', ':'),
    sort_keys=True,
    indent=4)


sub-017
[apply_slf]


100%|██████████| 2996/2996 [15:02<00:00,  3.32it/s]


[apply_slf]


100%|██████████| 16436/16436 [1:21:20<00:00,  3.37it/s]


In [ ]:
chanks_test = codecs.open(f'{constants.BASE_DATASET_PATH}/{subject}/X_test_chunks.json', 'r', encoding='utf-8').read()
chanks_test = json.loads(chanks_test)
chanks_test = np.array(chanks_test)
chanks_test.shape

(2996, 25, 100)

In [ ]:
y_test_chunks = codecs.open(file_path, 'r', encoding='utf-8').read()
y_test_chunks = json.loads(y_test_chunks)
y_test_chunks = np.array(y_test_chunks)
y_test_chunks.shape

(2996,)